In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [12]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocess the data

### Drop unimportant features

In [13]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

### Encode categorical variables

In [14]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

### Onehot encode 'Geography

In [15]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [16]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


### Train Test Split

In [17]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### Scale the data

In [18]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

### Save the encoders and scaler

In [19]:
with open('models_regression/label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open("models_regression/onehot_encoder_geo.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geo, file)

with open("models_regression/scaler.pkl", "wb") as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Define the ANN

In [21]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Set up Tensorboard

In [22]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

### Set up Early Stopping

In [23]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

### Train the model

In [24]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 3s 6ms/step - loss: 100389.6797 - mae: 100389.6797 - val_loss: 98565.6875 - val_mae: 98565.6875
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 99821.8047 - mae: 99821.8047 - val_loss: 97420.0312 - val_mae: 97420.0312
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 97825.8203 - mae: 97825.8203 - val_loss: 94478.7969 - val_mae: 94478.7969
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 93834.7109 - mae: 93834.7109 - val_loss: 89436.8984 - val_mae: 89436.8984
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 87812.4219 - mae: 87812.4219 - val_loss: 82625.5312 - val_mae: 82625.5312
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 80279.9688 - mae: 80279.9688 - val_loss: 74842.7969 - val_mae: 74842.7969
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 72229.7266 - mae: 72229.726

In [26]:
model.save("models_regression/model.h5")

c:\Projekte\Data_Science\annclassification\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Use Tensorboard

In [27]:
%load_ext tensorboard

In [30]:
%tensorboard --logdir regressionlogs/fit/20241030-152753

## Evaluate model on the test data

In [31]:
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 1ms/step - loss: 50371.7656 - mae: 50371.7656
Test MAE : 50371.765625
